In [111]:
import os
import sys
import copy
import json
import pickle

import numpy as np
import rmgpy.chemkin
import rmgpy.tools.uncertainty

import matplotlib.pyplot as plt
%matplotlib inline

In [59]:
def unpack_sensitivity(long_desc):
    start_str = 'sensitivities = '
    if start_str not in long_desc:
        return []
    start_index = long_desc.find(start_str) + len(start_str)
    sensitivities_str = long_desc[start_index:].replace("'", '"')
    sensitivities_str = sensitivities_str.replace("nan", '"-9999999"')
    sensitivities_str = sensitivities_str.replace('name', 'training_rxn_name')
    return json.loads(sensitivities_str)

In [2]:
# Load the RMG mechanism

# Load the base model and the covariance matrix
# basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model'
basedir = '/home/moon/autoscience/autoscience/butane/models/rmg_model'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)

# covariance_file = '/work/westgroup/harris.se/autoscience/autoscience/uncertainty/butane_covariance.pickle'
covariance_file = '/home/moon/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)


# Load the RMG Database

In [3]:
# load the model

uncertainty = rmgpy.tools.uncertainty.Uncertainty(output_directory='rmg_uncertainty')
uncertainty.load_model(base_chemkin, dictionary)




# TODO - force the user to provide the input file used to generate the mechanism to ensure databases are really the same
# load the database
# --------------- CAUTION!!! Databases here must match the ones used to generate the mechanism
# note - this cell stalls out on Discovery
thermo_libs = [
    'BurkeH2O2',
    'primaryThermoLibrary',
    'FFCM1(-)',
    'CurranPentane',
    'Klippenstein_Glarborg2016',
    'thermo_DFT_CCSDTF12_BAC',
    'DFT_QCI_thermo',
    'CBS_QB3_1dHR',
]

kinetic_libs = [
    'FFCM1(-)',
    'CurranPentane',
    'combustion_core/version5',
    'Klippenstein_Glarborg2016',
    'BurkeH2O2inArHe',
    'BurkeH2O2inN2',
]
uncertainty.load_database(
    thermo_libraries=thermo_libs,
    kinetics_families='default',
    reaction_libraries=kinetic_libs,
    kinetics_depositories=['training'],
)


In [8]:
# don't just count the nodes to get popularity. Use the sensitivity of each reaction

# start by loading sensitivity of reactions
rxn_sensitivity_file = '/home/moon/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/reaction_ignition_delay_plain.npy'
base_delay = 0.14857535
ignition_delay = np.load(rxn_sensitivity_file)
sensitivities = np.abs(ignition_delay - base_delay) / base_delay

In [5]:
# Get the different kinetic and thermo sources
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties()

In [6]:
# Create a giant dictionary with all of the reaction family information in it
auto_gen_families = {}
for family_name in uncertainty.database.kinetics.families.keys():
    if family_name == 'Intra_R_Add_Endocyclic' or family_name == 'Intra_R_Add_Exocyclic':
        continue
    if uncertainty.database.kinetics.families[family_name].auto_generated and family_name not in auto_gen_families.keys():
        auto_gen_families[family_name] = uncertainty.database.kinetics.families[family_name].rules.get_entries()
        auto_gen_families[f'{family_name}_labels'] = [entry.label for entry in uncertainty.database.kinetics.families[family_name].rules.get_entries()]
        auto_gen_families[f'{family_name}_rxn_map'] = uncertainty.database.kinetics.families[family_name].get_reaction_matches(
            thermo_database=uncertainty.database.thermo,
            remove_degeneracy=True,
            get_reverse=True,
            exact_matches_only=False,
            fix_labels=True)

## Just Look at Disproportionation

In [90]:
# Make a set of all the training reactions in the Disproportionation Family Tree
family_name = 'Disproportionation'
training_reactions = set()
for node_name in uncertainty.database.kinetics.families[family_name].rules.entries.keys():
    for training_rxn in auto_gen_families[f'{family_name}_rxn_map'][node_name]:
        training_reactions.add(training_rxn)
training_reactions = list(training_reactions)
print(len(training_reactions), 'training reactions')


137 training reactions


In [102]:
for training_reaction in training_reactions:
    print(training_reaction)

C3H7-2 + C2H5-2 <=> C2H6 + C3H6-2
HN2 + NO <=> HNO + N2
C5H7 + C2H5S <=> C5H8 + C2H4S-2
H + H2N2 <=> H2 + HN2
C3H5 + C3H7-2 <=> C3H6 + C3H6-2
H2N2 + O <=> HO-2 + HN2
C2H3 + O2 <=> C2H2_1 + HO2
C2H3S + C4H9-2 <=> C2H4S + C4H8-2
HO + H2N2 <=> H2O + HN2
H + C2H5 <=> H2 + C2H4
CH3O-2 + C3H7-2 <=> CH4O + C3H6-2
H2N2 + CH3_r1 <=> CH4 + HN2
C4H9-2 + CH3S <=> CH4S + C4H8-2
CH3_r1 + C2H5 <=> CH4 + C2H4
H2N + H2N2 <=> H3N + HN2
C2H5 + C2H5-2 <=> C2H6 + C2H4
C3H7-2 + C3H7 <=> C3H8 + C3H6-2
HO2-2 + H2N2 <=> H2O2 + HN2
C3H5 + C2H5 <=> C3H6 + C2H4
C3H7-2 + C4H9 <=> C4H10 + C3H6-2
H + H3N2 <=> H2 + H2N2-2
CH3O-2 + C2H5 <=> CH4O + C2H4
C2H3-2 + C3H7-2 <=> C2H4-2 + C3H6-2
H3N2 + O <=> HO-2 + H2N2-2
C2H5 + C3H7 <=> C3H8 + C2H4
HS2 + C5H9 <=> H2S2 + C5H8-2
HO + H3N2 <=> H2O + H2N2-2
C2H + CH3O <=> C2H2 + CH2O
C2H5 + C4H9 <=> C4H10 + C2H4
C2H + C3H7-2 <=> C2H2 + C3H6-2
H3N2 + CH3_r1 <=> CH4 + H2N2-2
C4H9-2 + C4H9 <=> C4H10 + C4H8-2
CH3O + C2H3-2 <=> C2H4-2 + CH2O
H + H2NO-3 <=> H2 + HNO-3
CH4N + CH3_r1 <=

In [153]:
training_reaction_scores = np.zeros(len(training_reactions))
for i, reaction in enumerate(uncertainty.reaction_list):
#     reaction_sensitivity = 1.0
    reaction_sensitivity = sensitivities[i]

    try:
        if reaction.family == family_name:
            # Get the node
            node_name = ''
#                 print(i, reaction, node_name)
            print(i, reaction)
            if 'Rate Rules' in uncertainty.reaction_sources_dict[uncertainty.reaction_list[i]].keys():

                node_name = uncertainty.reaction_sources_dict[uncertainty.reaction_list[i]]['Rate Rules'][1]['node']
                node_sensitivities = unpack_sensitivity(uncertainty.database.kinetics.families['Disproportionation'].rules.entries[node_name][0].long_desc)
                    
                # find each training reaction in the node
                for t, train in enumerate(auto_gen_families[f'{family_name}_rxn_map'][node_name]):
                    # Get the partial sensitivity dNode / dtraining_reaction (A factor)
                    # equal division
                    partial_sensitivity = 1.0 / len(auto_gen_families[f'{family_name}_rxn_map'][node_name])
                    
                    # use calculated derivatives
#                     partial_sensitivity = node_sensitivities[t]['dA']
                    # use the chain rule and multiply by this mechanism reaction's sensitivity
                    total_sensitivity = reaction_sensitivity * partial_sensitivity
                    
                    for j, training_rxn in enumerate(training_reactions):
                        if training_rxn.is_isomorphic(train):
                            training_reaction_scores[j] += total_sensitivity  # or whatever the reaction sensitivity is
                            print('\t', training_rxn, '\t', np.round(total_sensitivity, 8))
                            break
                    else:
                        raise ValueError('could not match training reaction in node')
                

            elif 'Training' in uncertainty.reaction_sources_dict[uncertainty.reaction_list[i]].keys():
                for j, training_rxn in enumerate(training_reactions):
                    if training_rxn.is_isomorphic(uncertainty.reaction_sources_dict[uncertainty.reaction_list[i]]['Training'][1].item):
                        training_reaction_scores[j] += reaction_sensitivity  # or whatever the reaction sensitivity is
                        print('matched training reaction exactly!')
                        break
            print()
            
    except AttributeError:
        continue


213 HO2(16) + C2H5(33) <=> H2O2(17) + C2H4(11)
	 HO2-2 + HN2 <=> H2O2 + N2 	 1.389e-05
	 HO2-2 + H2N2 <=> H2O2 + HN2 	 1.389e-05
	 HO2-2 + H3N2 <=> H2O2 + H2N2-2 	 1.389e-05
	 HO2-2 + H2NO <=> H2O2 + HNO-2 	 1.389e-05
	 HO2-2 + CH2N <=> H2O2 + CHN 	 1.389e-05

215 CH2(23) + C2H5(33) <=> CH3(18) + C2H4(11)
matched training reaction exactly!

216 C2H3(22) + CH3CO(20) <=> CH2CO(24) + C2H4(11)
	 C4H7-2 + O2 <=> HO2 + C4H6 	 0.0
	 H + CH2NO <=> H2 + CHNO 	 0.0
	 HO + CH2NO <=> H2O + CHNO 	 0.0
	 CH3_r1 + CH2NO <=> CH4 + CHNO 	 0.0
	 H2N + CH2NO <=> H3N + CHNO 	 0.0
	 C5H7 + C4H7-2 <=> C5H8 + C4H6 	 0.0

217 C2H3(22) + CH2CHO(21) <=> CH2CO(24) + C2H4(11)
	 CH2 + C3H7-2 <=> CH3 + C3H6-2 	 0.0
	 C2H3S + C3H7-2 <=> C2H4S + C3H6-2 	 0.0
	 H + C3H7-2 <=> H2 + C3H6-2 	 0.0
	 C4H7 + C2H5S <=> C4H8 + C2H4S-2 	 0.0
	 CH3_r1 + C3H7-2 <=> CH4 + C3H6-2 	 0.0
	 C3H7-2 + C2H5-2 <=> C2H6 + C3H6-2 	 0.0
	 C5H7 + C2H5S <=> C5H8 + C2H4S-2 	 0.0
	 C3H5 + C3H7-2 <=> C3H6 + C3H6-2 	 0.0
	 C2H3S + C4H9-2 <=> C2H4

422 HO2(16) + SC4H9(183) <=> H2O2(17) + C4H8(188)
	 C4H7-2 + O2 <=> HO2 + C4H6 	 3.554e-05

423 HCO(19) + CH3CO(20) <=> CH2O(9) + CH2CO(24)
	 C4H7-2 + O2 <=> HO2 + C4H6 	 1e-08
	 H + CH2NO <=> H2 + CHNO 	 1e-08
	 HO + CH2NO <=> H2O + CHNO 	 1e-08
	 CH3_r1 + CH2NO <=> CH4 + CHNO 	 1e-08
	 H2N + CH2NO <=> H3N + CHNO 	 1e-08
	 C5H7 + C4H7-2 <=> C5H8 + C4H6 	 1e-08

424 HCO(19) + CH2CHO(21) <=> CH2O(9) + CH2CO(24)
	 CH2 + C3H7-2 <=> CH3 + C3H6-2 	 0.0
	 C2H3S + C3H7-2 <=> C2H4S + C3H6-2 	 0.0
	 H + C3H7-2 <=> H2 + C3H6-2 	 0.0
	 C4H7 + C2H5S <=> C4H8 + C2H4S-2 	 0.0
	 CH3_r1 + C3H7-2 <=> CH4 + C3H6-2 	 0.0
	 C3H7-2 + C2H5-2 <=> C2H6 + C3H6-2 	 0.0
	 C5H7 + C2H5S <=> C5H8 + C2H4S-2 	 0.0
	 C3H5 + C3H7-2 <=> C3H6 + C3H6-2 	 0.0
	 C2H3S + C4H9-2 <=> C2H4S + C4H8-2 	 0.0
	 CH3O-2 + C3H7-2 <=> CH4O + C3H6-2 	 0.0
	 C4H9-2 + CH3S <=> CH4S + C4H8-2 	 0.0
	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2 	 0.0
	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2 	 0.0
	 C2H3-2 + C3H7-2 <=> C2H4-2 + C3H6-2 	 0.0
	 HS2 + C5H9 <=> H2

	 C4H9-2 + C3H7 <=> C3H8 + C4H8-2 	 1e-08
	 C4H9-2 + C4H9 <=> C4H10 + C4H8-2 	 1e-08
	 C2H3-2 + C4H9-2 <=> C2H4-2 + C4H8-2 	 1e-08
	 CH3_r1 + C3H5-2 <=> CH4 + C3H4 	 1e-08
	 C3H5-2 + C2H5-2 <=> C2H6 + C3H4 	 1e-08
	 C3H5 + C3H5-2 <=> C3H6 + C3H4 	 1e-08
	 C3H5-2 + C3H7 <=> C3H8 + C3H4 	 1e-08
	 C3H5-2 + C4H9 <=> C4H10 + C3H4 	 1e-08
	 C2H3-2 + C3H5-2 <=> C2H4-2 + C3H4 	 1e-08
	 C2H5S-2 + C3H7-2 <=> C2H6S + C3H6-2 	 1e-08
	 C6H5 + C6H5-2 <=> C6H6 + C6H4 	 1e-08

468 CH2(23) + CH3CO(20) <=> CH3(18) + CH2CO(24)
	 C4H7-2 + O2 <=> HO2 + C4H6 	 0.0
	 H + CH2NO <=> H2 + CHNO 	 0.0
	 HO + CH2NO <=> H2O + CHNO 	 0.0
	 CH3_r1 + CH2NO <=> CH4 + CHNO 	 0.0
	 H2N + CH2NO <=> H3N + CHNO 	 0.0
	 C5H7 + C4H7-2 <=> C5H8 + C4H6 	 0.0

469 CH2(23) + CH2CHO(21) <=> CH3(18) + CH2CO(24)
	 CH2 + C3H7-2 <=> CH3 + C3H6-2 	 0.0
	 C2H3S + C3H7-2 <=> C2H4S + C3H6-2 	 0.0
	 H + C3H7-2 <=> H2 + C3H6-2 	 0.0
	 C4H7 + C2H5S <=> C4H8 + C2H4S-2 	 0.0
	 CH3_r1 + C3H7-2 <=> CH4 + C3H6-2 	 0.0
	 C3H7-2 + C2H5-2 <=> C2H6 +

574 CH3O(31) + C3H5-A(94) <=> CH2O(9) + C3H6(12)
	 C2H5 + C2H5-2 <=> C2H6 + C2H4 	 1.1e-07
	 C3H5 + C2H5 <=> C3H6 + C2H4 	 1.1e-07
	 CH3O-2 + C2H5 <=> CH4O + C2H4 	 1.1e-07
	 C2H5 + C3H7 <=> C3H8 + C2H4 	 1.1e-07
	 C2H5 + C4H9 <=> C4H10 + C2H4 	 1.1e-07
	 C2H3-2 + C2H5 <=> C2H4-2 + C2H4 	 1.1e-07
	 HS2 + CH3S-2 <=> H2S2 + CH2S 	 1.1e-07
	 CH3S + CH3S-3 <=> CH4S + CH2S-2 	 1.1e-07
	 HN2 + NO <=> HNO + N2 	 1.1e-07

575 C3H5-A(94) + NC3H7(92) <=> C3H6(12) + C3H6(12)
matched training reaction exactly!

576 C3H5-A(94) + IC3H7(93) <=> C3H6(12) + C3H6(12)
	 C5H7 + C4H7-2 <=> C5H8 + C4H6 	 2e-08

577 C3H5-A(94) + SC4H9(183) <=> C3H6(12) + C4H8(189)
	 C3H5 + C3H7-2 <=> C3H6 + C3H6-2 	 3.39e-06

578 C3H5-A(94) + PC4H9(182) <=> C3H6(12) + C4H8(188)
	 C3H5 + C3H7-2 <=> C3H6 + C3H6-2 	 1.33e-06

579 C3H5-A(94) + SC4H9(183) <=> C3H6(12) + C4H8(188)
	 C5H7 + C4H7-2 <=> C5H8 + C4H6 	 1.5e-07

580 CH3CO(20) + C3H5-A(94) <=> CH2CO(24) + C3H6(12)
	 C4H7-2 + O2 <=> HO2 + C4H6 	 0.0
	 H + CH2NO <=> H2 + C

	 C3H5 + C3H7-2 <=> C3H6 + C3H6-2 	 0.0
	 CH3O-2 + C3H7-2 <=> CH4O + C3H6-2 	 0.0
	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2 	 0.0
	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2 	 0.0
	 C2H3-2 + C3H7-2 <=> C2H4-2 + C3H6-2 	 0.0
	 C6H5 + C6H5-2 <=> C6H6 + C6H4 	 0.0

632 CH2CHO(21) + C2H4O(703) <=> CH2CHO(21) + CH3CHO(35)
	 C4H7 + C2H5S <=> C4H8 + C2H4S-2 	 0.0
	 C3H7-2 + C2H5-2 <=> C2H6 + C3H6-2 	 0.0
	 C5H7 + C2H5S <=> C5H8 + C2H4S-2 	 0.0
	 C3H5 + C3H7-2 <=> C3H6 + C3H6-2 	 0.0
	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2 	 0.0
	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2 	 0.0
	 C2H3-2 + C3H7-2 <=> C2H4-2 + C3H6-2 	 0.0

633 CH3CO(20) + C2H4O(703) <=> CH2CHO(21) + CH3CHO(35)
	 C4H7 + C2H5S <=> C4H8 + C2H4S-2 	 1e-08
	 C5H7 + C2H5S <=> C5H8 + C2H4S-2 	 1e-08
	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2 	 1e-08
	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2 	 1e-08

634 C2H3(22) + C2H4O(703) <=> CH2CHO(21) + C2H4(11)
	 C4H7 + C2H5S <=> C4H8 + C2H4S-2 	 0.0
	 C3H7-2 + C2H5-2 <=> C2H6 + C3H6-2 	 0.0
	 C5H7 + C2H5S <=> C5H8 + C2H4S-2 	 0.0
	 C3H5 + C3

...

	 C5H7 + C2H5S <=> C5H8 + C2H4S-2 	 0.0
	 C3H5 + C3H7-2 <=> C3H6 + C3H6-2 	 0.0
	 C2H3S + C4H9-2 <=> C2H4S + C4H8-2 	 0.0
	 CH3O-2 + C3H7-2 <=> CH4O + C3H6-2 	 0.0
	 C4H9-2 + CH3S <=> CH4S + C4H8-2 	 0.0
	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2 	 0.0
	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2 	 0.0
	 C2H3-2 + C3H7-2 <=> C2H4-2 + C3H6-2 	 0.0
	 HS2 + C5H9 <=> H2S2 + C5H8-2 	 0.0
	 C2H + C3H7-2 <=> C2H2 + C3H6-2 	 0.0
	 C2H + C4H9-2 <=> C2H2 + C4H8-2 	 0.0
	 H + C4H9-2 <=> H2 + C4H8-2 	 0.0
	 CH3_r1 + C4H9-2 <=> CH4 + C4H8-2 	 0.0
	 C4H9-2 + C2H5-2 <=> C2H6 + C4H8-2 	 0.0
	 CH3O-2 + C4H9-2 <=> CH4O + C4H8-2 	 0.0
	 C3H5 + C4H9-2 <=> C3H6 + C4H8-2 	 0.0
	 C4H9-2 + C3H7 <=> C3H8 + C4H8-2 	 0.0
	 C4H9-2 + C4H9 <=> C4H10 + C4H8-2 	 0.0
	 C2H3-2 + C4H9-2 <=> C2H4-2 + C4H8-2 	 0.0
	 CH3_r1 + C3H5-2 <=> CH4 + C3H4 	 0.0
	 C3H5-2 + C2H5-2 <=> C2H6 + C3H4 	 0.0
	 C3H5 + C3H5-2 <=> C3H6 + C3H4 	 0.0
	 C3H5-2 + C3H7 <=> C3H8 + C3H4 	 0.0
	 C3H5-2 + C4H9 <=> C4H10 + C3H4 	 0.0
	 C2H3-2 + C3H5-2 <=> C2H4-2 + C3H4 	

	 CH3_r1 + C3H5-2 <=> CH4 + C3H4 	 0.0
	 C3H5-2 + C2H5-2 <=> C2H6 + C3H4 	 0.0
	 C3H5 + C3H5-2 <=> C3H6 + C3H4 	 0.0
	 C3H5-2 + C3H7 <=> C3H8 + C3H4 	 0.0
	 C3H5-2 + C4H9 <=> C4H10 + C3H4 	 0.0
	 C2H3-2 + C3H5-2 <=> C2H4-2 + C3H4 	 0.0
	 HO + C3H5-2 <=> H2O + C3H4 	 0.0
	 C2H5S-2 + C3H7-2 <=> C2H6S + C3H6-2 	 0.0
	 C4H7-3 + O2 <=> HO2 + C4H6-2 	 0.0
	 C3H7-2 + O2 <=> HO2 + C3H6-2 	 0.0
	 C4H7-3 + O2 <=> HO2 + C4H6-2 	 0.0
	 C4H9-2 + O2 <=> HO2 + C4H8-2 	 0.0
	 C5H9-2 + O2 <=> HO2 + C5H8-3 	 0.0
	 C6H9 + O2 <=> HO2 + C6H8 	 0.0
	 C6H5 + C6H5-2 <=> C6H6 + C6H4 	 0.0

1582 OCHO(38) + C2H3(22) <=> HOCHO(40) + C2H2(25)
	 HO + CH2N <=> H2O + CHN 	 4e-08

1584 OCHO(38) + OCHO(38) <=> CO2(7) + HOCHO(40)
	 C3H7-2 + O2 <=> HO2 + C3H6-2 	 0.0
	 CH2 + C3H7-2 <=> CH3 + C3H6-2 	 0.0
	 C2H3S + C3H7-2 <=> C2H4S + C3H6-2 	 0.0
	 H + C3H7-2 <=> H2 + C3H6-2 	 0.0
	 C4H7 + C2H5S <=> C4H8 + C2H4S-2 	 0.0
	 CH3_r1 + C3H7-2 <=> CH4 + C3H6-2 	 0.0
	 C3H7-2 + C2H5-2 <=> C2H6 + C3H6-2 	 0.0
	 C5H7 + C2H5S <=> C

	 C4H9-2 + O2 <=> HO2 + C4H8-2 	 0.0
	 C2H + C4H9-2 <=> C2H2 + C4H8-2 	 0.0
	 H + C4H9-2 <=> H2 + C4H8-2 	 0.0
	 CH3_r1 + C4H9-2 <=> CH4 + C4H8-2 	 0.0
	 C4H9-2 + C2H5-2 <=> C2H6 + C4H8-2 	 0.0
	 CH3O-2 + C4H9-2 <=> CH4O + C4H8-2 	 0.0
	 C3H5 + C4H9-2 <=> C3H6 + C4H8-2 	 0.0
	 C4H9-2 + C3H7 <=> C3H8 + C4H8-2 	 0.0
	 C4H9-2 + C4H9 <=> C4H10 + C4H8-2 	 0.0
	 C2H3-2 + C4H9-2 <=> C2H4-2 + C4H8-2 	 0.0
	 HO + C4H9-2 <=> H2O + C4H8-2 	 0.0
	 C3H5-2 + O2 <=> HO2 + C3H4 	 0.0
	 CH3_r1 + C3H5-2 <=> CH4 + C3H4 	 0.0
	 C3H5-2 + C2H5-2 <=> C2H6 + C3H4 	 0.0
	 C3H5 + C3H5-2 <=> C3H6 + C3H4 	 0.0
	 C3H5-2 + C3H7 <=> C3H8 + C3H4 	 0.0
	 C3H5-2 + C4H9 <=> C4H10 + C3H4 	 0.0
	 C2H3-2 + C3H5-2 <=> C2H4-2 + C3H4 	 0.0
	 HO + C3H5-2 <=> H2O + C3H4 	 0.0
	 C2H5S-2 + C3H7-2 <=> C2H6S + C3H6-2 	 0.0
	 C4H7-3 + O2 <=> HO2 + C4H6-2 	 0.0
	 C3H7-2 + O2 <=> HO2 + C3H6-2 	 0.0
	 C4H7-3 + O2 <=> HO2 + C4H6-2 	 0.0
	 C4H9-2 + O2 <=> HO2 + C4H8-2 	 0.0
	 C5H9-2 + O2 <=> HO2 + C5H8-3 	 0.0
	 C6H9 + O2 <=> HO2 + C6H8 

	 C2H3-2 + C3H7-2 <=> C2H4-2 + C3H6-2 	 0.0
	 HS2 + C5H9 <=> H2S2 + C5H8-2 	 0.0
	 C2H + C3H7-2 <=> C2H2 + C3H6-2 	 0.0
	 HO + C3H7-2 <=> H2O + C3H6-2 	 0.0
	 C4H9-2 + O2 <=> HO2 + C4H8-2 	 0.0
	 C2H + C4H9-2 <=> C2H2 + C4H8-2 	 0.0
	 H + C4H9-2 <=> H2 + C4H8-2 	 0.0
	 CH3_r1 + C4H9-2 <=> CH4 + C4H8-2 	 0.0
	 C4H9-2 + C2H5-2 <=> C2H6 + C4H8-2 	 0.0
	 CH3O-2 + C4H9-2 <=> CH4O + C4H8-2 	 0.0
	 C3H5 + C4H9-2 <=> C3H6 + C4H8-2 	 0.0
	 C4H9-2 + C3H7 <=> C3H8 + C4H8-2 	 0.0
	 C4H9-2 + C4H9 <=> C4H10 + C4H8-2 	 0.0
	 C2H3-2 + C4H9-2 <=> C2H4-2 + C4H8-2 	 0.0
	 HO + C4H9-2 <=> H2O + C4H8-2 	 0.0
	 C3H5-2 + O2 <=> HO2 + C3H4 	 0.0
	 CH3_r1 + C3H5-2 <=> CH4 + C3H4 	 0.0
	 C3H5-2 + C2H5-2 <=> C2H6 + C3H4 	 0.0
	 C3H5 + C3H5-2 <=> C3H6 + C3H4 	 0.0
	 C3H5-2 + C3H7 <=> C3H8 + C3H4 	 0.0
	 C3H5-2 + C4H9 <=> C4H10 + C3H4 	 0.0
	 C2H3-2 + C3H5-2 <=> C2H4-2 + C3H4 	 0.0
	 HO + C3H5-2 <=> H2O + C3H4 	 0.0
	 C2H5S-2 + C3H7-2 <=> C2H6S + C3H6-2 	 0.0
	 C4H7-3 + O2 <=> HO2 + C4H6-2 	 0.0
	 C3H7-2 + O2 <=> 

	 H + C4H9-2 <=> H2 + C4H8-2 	 0.0
	 CH3_r1 + C4H9-2 <=> CH4 + C4H8-2 	 0.0
	 C4H9-2 + C2H5-2 <=> C2H6 + C4H8-2 	 0.0
	 CH3O-2 + C4H9-2 <=> CH4O + C4H8-2 	 0.0
	 C3H5 + C4H9-2 <=> C3H6 + C4H8-2 	 0.0
	 C4H9-2 + C3H7 <=> C3H8 + C4H8-2 	 0.0
	 C4H9-2 + C4H9 <=> C4H10 + C4H8-2 	 0.0
	 C2H3-2 + C4H9-2 <=> C2H4-2 + C4H8-2 	 0.0
	 CH3_r1 + C3H5-2 <=> CH4 + C3H4 	 0.0
	 C3H5-2 + C2H5-2 <=> C2H6 + C3H4 	 0.0
	 C3H5 + C3H5-2 <=> C3H6 + C3H4 	 0.0
	 C3H5-2 + C3H7 <=> C3H8 + C3H4 	 0.0
	 C3H5-2 + C4H9 <=> C4H10 + C3H4 	 0.0
	 C2H3-2 + C3H5-2 <=> C2H4-2 + C3H4 	 0.0
	 C2H5S-2 + C3H7-2 <=> C2H6S + C3H6-2 	 0.0
	 C6H5 + C6H5-2 <=> C6H6 + C6H4 	 0.0

1759 C2H3(22) + S(186) <=> C2H2(25) + S(187)
	 HO2-2 + HN2 <=> H2O2 + N2 	 1e-08
	 HO2-2 + H2N2 <=> H2O2 + HN2 	 1e-08
	 HO2-2 + H3N2 <=> H2O2 + H2N2-2 	 1e-08
	 HO2-2 + H2NO <=> H2O2 + HNO-2 	 1e-08
	 HO2-2 + CH2N <=> H2O2 + CHN 	 1e-08

1760 C2H3(22) + S(223) <=> C2H2(25) + S(187)
	 C2H5 + C2H5-2 <=> C2H6 + C2H4 	 1e-08
	 C3H5 + C2H5 <=> C3H6 + C2H4 	 

In [154]:
# print the results
order = np.arange(0, len(training_reaction_scores))
train_rxn_order = [x for _, x in sorted(zip(training_reaction_scores, order))][::-1]


print('\tTraining Reaction\t\tSensitivity')
for i in range(0, len(training_reaction_scores)):
    index = train_rxn_order[i]
    print(index,'\t', training_reactions[index], '\t', training_reaction_scores[index])



	Training Reaction		Sensitivity
103 	 C4H7-2 + O2 <=> HO2 + C4H6 	 0.00011378094261052124
130 	 HO2-2 + HN2 <=> H2O2 + N2 	 1.624836322898186e-05
121 	 HO2-2 + H2NO <=> H2O2 + HNO-2 	 1.624836322898186e-05
104 	 HO2-2 + CH2N <=> H2O2 + CHN 	 1.624836322898186e-05
49 	 HO2-2 + H3N2 <=> H2O2 + H2N2-2 	 1.624836322898186e-05
17 	 HO2-2 + H2N2 <=> H2O2 + HN2 	 1.624836322898186e-05
4 	 C3H5 + C3H7-2 <=> C3H6 + C3H6-2 	 1.297059384486081e-05
114 	 C4H7-3 + O2 <=> HO2 + C4H6-2 	 9.339231175494864e-06
75 	 C3H7-2 + O2 <=> HO2 + C3H6-2 	 8.800149175367438e-06
38 	 C4H9-2 + O2 <=> HO2 + C4H8-2 	 8.800149175367438e-06
19 	 C3H7-2 + C4H9 <=> C4H10 + C3H6-2 	 7.352757144253404e-06
127 	 CH3_r1 + C3H7-2 <=> CH4 + C3H6-2 	 7.2998539467376545e-06
16 	 C3H7-2 + C3H7 <=> C3H8 + C3H6-2 	 7.2349300005097024e-06
126 	 C5H7 + C4H7-2 <=> C5H8 + C4H6 	 7.1171403647485555e-06
58 	 C6H9 + O2 <=> HO2 + C6H8 	 4.669615587747436e-06
48 	 C5H9-2 + O2 <=> HO2 + C5H8-3 	 4.669615587747436e-06
46 	 HO + C3H7-2 <=> H2